In [2]:
import numpy as np
from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [3]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ... ENTER YOUR CODE HERE ...
    # get the size of vec
    size = len(vec)

    # create v1
    e = np.zeros_like(vec).astype(float)
    e[0] = 1.0
    v1 = e * np.linalg.norm(vec)

    # create H
    u = vec - v1
    u = u / np.linalg.norm(u)
    H = np.identity(size) - 2 * (np.outer(u, u))

    return v1, H

Test your function using tests below:

In [4]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol = 1e-10)
assert_allclose(np.dot(h, v), v1, atol = 1e-10)

In [5]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec, atol = 1e-10)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [6]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # implementation
    R = a1.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        # Apply Householder transformation.
        normx = np.linalg.norm(R[j:, j])
        if R[j, j] > 0:
            u1 = R[j, j] + normx
            tau = u1 / normx
        else:
            u1 = R[j, j] - normx
            tau = -u1 / normx 
        # print(u1)
        # print(tau)
        w = R[j:, j] / u1
        w[0] = 1
        w = np.array([w])

        # print(w.T)
        # print(R[j:, :])
        R[j:, :] = R[j:, :] - (tau*w.T)@(w@R[j:, :])
        Q[:, j:] = Q[:, j:] - ((Q[:, j:]@w.T)@np.transpose(tau*w.T))
    
    return Q, R



In [7]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [8]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a, atol=1e-10)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [9]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

Q and R should agree with qq and rr because they are doing QR decomposition through a loop doing Householder reflections.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [10]:
# Compute the QR decomposition of an m-by-n matrix A using
# Householder transformations, re-using the storage of A for the Q and R factors.
def qr_decomp_v2(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # implementation
    R = a1.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        # Apply Householder transformation.
        normx = np.linalg.norm(R[j:, j])
        if R[j, j] > 0:
            u1 = R[j, j] + normx
            tau = u1 / normx
        else:
            u1 = R[j, j] - normx
            tau = -u1 / normx 
        # print(u1)
        # print(tau)
        w = R[j:, j] / u1
        w[0] = 1
        w = np.array([w])

        # print(w.T)
        # print(R[j:, :])
        R[j:, :] = R[j:, :] - (tau*w.T)@(w@R[j:, :])
        Q[:, j:] = Q[:, j:] - ((Q[:, j:]@w.T)@np.transpose(tau*w.T))
    
    return Q, R




In [11]:
# test Q@R = qq@rr
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
Q, R = qr_decomp_v2(a)
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(Q, R), np.dot(qq, rr))

In [12]:
# Use reflection vectors to multiply a matrix with the transpose of Q
# QR is the result of the first function
# X is the arbitrary matrix
def QTX(QR, tau, X):
    m, n = QR.shape
    QTX = X
    for j in range(0, n):
        w = [0, QR[j+1:,j]]
        w = np.array([w])
        QTX[j:,:] = QTX[j:,:]-[tau(j)@w.T]@[w@QTX[j:,:]]
    return QTX

In [13]:
# test Q.T@a = R
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
Q, R = qr_decomp_v2(a)

assert_allclose(np.dot(Q.T, a), R, atol=1e-10)